In [ ]:
import numpy as np
import pandas as pd

#precisao de casas decimais
pd.set_option("display.precision", 3)
#maximo de colunas
pd.set_option("display.max_columns", 200)
#maximo de linhas
pd.set_option("display.max_rows", 200)

In [ ]:
df = pd.read_csv('./incident_event_log.csv', low_memory=False)
df.head(200)

In [ ]:
#ajustando notify
df['notify'] = df['notify'].map({'Do Not Notify': False, 'Send Email': True})

In [ ]:
#selecionando colunas que se repetem nas linhas referentes ao mesmo incidente
common_columns = np.array(['number', 'caller_id', 'opened_by', 'opened_at', 'sys_created_by', 'sys_created_at', 'contact_type', 'notify', 'closed_code', 'resolved_at', 'closed_at'])
#agrupando em um novo df
grouped_df = df.groupby(by=list(common_columns))

In [ ]:
df.describe()

In [ ]:
df.describe(include=['object'])